# Using Cuxfilter to Plot Coordinates on a Map of the United States
### A map of the US households and tanks with a distance and earthquake risk slider
This visualization uses a the Cuxfilter library and GPUs to plot households and tanks on a dashboard, with range slider feature to show households that have a certain risk index range. There is also a range slider for distance between the houeshold and its nearest tank, and a multiselect to view only houses with or without elderly people.

### Import statements

In [19]:
import pandas as pd
import os

import cuxfilter
from cuxfilter.layouts import feature_and_five_edge, double_feature
import cudf

### Setting ```DATA_DIR```
In order to read in files from this repository, we must set ```DATA_DIR``` to be the data folder within this repository. This requires ```os.getcwd()``` to return the path to the processing notebook of this repository, so ```xxx/codeplus-celine-dcc-package/visualizations/07_natural_hazards```, where ```xxx``` is the path to where you cloned this repository. If it is not, use ```os.chdir(path)``` to change the current working directory to ```xxx/codeplus-celine-dcc-package/visualizations/07_natural_hazards``` before getting the current working directory in ```DATA_DIR = os.getcwd()```, where ```path``` is ```xxx/codeplus-celine-dcc-package/visualizations/07_natural_hazards```.

In [20]:
DATA_DIR = os.getcwd()
DATA_DIR = DATA_DIR.replace('visualizations/07_natural_hazards', 'data')
DATA_DIR

'/hpc/home/at341/ondemand/codeplus-celine-dcc-package/data'

### Reading earthquake risk and distance data

This is a preprocessed parquet file that includes the latitude and longitude of households and tanks that are already transformed into the 3857 projections. This file also contains the ```is_elderly``` variable that indicates if the point at a particular lat/lon is a household with elderly, a household without elderly, or a tank. ```distance_mi``` is the distance in miles between tanks and houses. It was created in processing notebook **08_nat_hazards_processing**.

In [21]:
df = pd.read_parquet(DATA_DIR + '/earthquake_risk.parquet')
df

,is_elderly,distance_mi,earthquake_risk,lat_3857,lon_3857
0,2,14.977140,7.975933,-8.556472e+06,4.754685e+06
1,2,812.480090,6.897670,-1.076073e+07,5.469166e+06
2,2,934.400050,0.509994,-1.251261e+07,4.094840e+06
3,1,509.132686,1.289903,-9.857755e+06,4.129311e+06
4,2,48.826977,2.205648,-9.267351e+06,5.493176e+06
...,...,...,...,...,...
73086,0,3200.000000,1.575536,-1.010035e+07,5.222881e+06
73087,0,3200.000000,3.312025,-1.183249e+07,5.291041e+06
73088,0,3200.000000,17.807754,-9.971313e+06,4.384699e+06
73089,0,3200.000000,9.400549,-7.944992e+06,5.135812e+06


### Transforming the dataframe type

Here, we are transforming the pandas dataframe into a cudf dataframe and then into a cuxfilter dataframe so that we can plot the points using the cuxfilter library.

In [22]:
cdf = cudf.DataFrame.from_pandas(df) 

In [23]:
cux_df = cuxfilter.DataFrame.from_dataframe(cdf) 

### Creating multiselect label map and defining map color palette

In the label map for the ```is_elderly``` multiselect, we are defining the labels for the values that equal 0, 1, and 2 respectively.

In [24]:
label_elderly = {0: 'Tank', 1: 'Elderly', 
             2: 'No Elderly'}

from bokeh.palettes import Spectral11
colors = list(Spectral11)
colors.reverse()


### Defining charts

Below, we are defining the four charts that will be plotted on our dashboard. In each chart, you can define the type of cuxfilter chart (```scatter```, ```multiselect```, ```range_slider```, ```drop_down```, etc.) and also specify the column that the chart is made on. The label map parameter specifies which label map the feature will pull from. 

In [25]:
household = cuxfilter.charts.scatter(x='lat_3857', y='lon_3857', pixel_shade_type='linear', color_palette = colors, aggregate_fn = 'max', aggregate_col = 'distance_mi', tile_provider="CartoDark", title = 'Households and distances to nearest tank, with earthquake risk slider', legend = True)

earthquake_risk= cuxfilter.charts.range_slider('earthquake_risk')

dist = cuxfilter.charts.range_slider('distance_mi')


### Plotting the charts on the dashboard

The layout and sidebar parameter determines how the charts are oganized on the dashboard. Here we have put three charts as the sidebar, however, you could also place those three charts underneath the main chart by specifying feature and base under the layout feature.

In [26]:
d = cux_df.dashboard([household], sidebar = [elderly, earthquake_risk, dist], 
                     layout = cuxfilter.layouts.feature_and_base, theme = cuxfilter.themes.rapids) 

### Display the visualization

In [27]:
d.show()
d.app(sidebar_width = 250)

Dashboard running at port 40355


Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_height', width=250)
        [0] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='Datapoints Selected')
            [0] Column(sizing_mode='stretch_both')
                [0] Number(css_classes=['indicator'], font_size='18pt', format='{value:,}', sizing_mode='stretch_width', value=73091)
                [1] Progress(sizing_mode='stretch_width', value=100)
        [1] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='is_elderly_multi_select', width=400)
            [0] MultiSelect(height=200, options={'Tank': 0, 'Elderly': 1, ...}, sizing_mode='stretch_both', value=[''], width=400)
        [2] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='earthquake_risk_range_sli..., width=400)
            [0] RangeSlider(end=100.0, sizing_mode='stretch_both', start=-1.0, value=(-1.0, 100.0), value_end=100.0, value_start=-1.0)
        [3] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='distance_mi_range_slider'..., width=400)
            [0] RangeSlider(end=3200.0, sizing_mode='stretch_both', start=4.25222194461619, value=(4.25222194461619, ..., value_end=3200.0, value_start=4.25222194461619)
    [1] GridStack(allow_drag=False, allow_resize=False, ncols=11, nrows=4, sizing_mode='stretch_both')
        [0] Card(sizing_mode='stretch_both', title='Households and distances ..., width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')